# Create virtual dataset referencing a collection of rechunked NetCDF datasets
Using rechunker.   Tried using Virtualizarr but it bombs

In [ ]:
import zarr
import fsspec
from dotenv import load_dotenv
import xarray as xr
import dask
from dask.distributed import LocalCluster, Client

In [ ]:
import kerchunk.combine
import kerchunk.zarr
from kerchunk.hdf import SingleHdf5ToZarr
from kerchunk.combine import MultiZarrToZarr
from fsspec.implementations.reference import LazyReferenceMapper

In [ ]:
import os
if 'NEBARI_JUPYTERHUB_SSH_PORT' in os.environ.keys():   # NEBARI
    dot_env_file = '/shared/users/nebari-setup/lib/CMRE_Blob_Keys.env'
elif 'AML_CloudName' in os.environ.keys():  # AzureML
    dot_env_file = '/home/azureuser/cloudfiles/code/Users/CMRE_Blob_Keys.env'
else:
    print('Unknown Compute Environment: Blob_Keys not found')
print(dot_env_file)
_ = load_dotenv(dot_env_file)  # loads Azure credentials

In [ ]:
storage_options={
            "account_name": os.environ["AZURE_STORAGE_CONNECTION_STRING"],
            "account_key": os.environ["AZURE_STORAGE_KEY"],
        }

In [ ]:
fs = fsspec.filesystem('abfs', storage_options=storage_options, use_listings_cache=False, skip_instance_cache=True)

In [ ]:
region = 'usa'
flist = fs.glob(f'abfs://cmre-cc01/data/era5/era5_{region}_??.nc')

In [ ]:
flist = [f'abfs://{f}' for f in flist]
print(len(flist))
print(flist[0])
print(flist[-1])

In [ ]:
opts = storage_options
so=dict(remote_protocol='abfs', remote_options=opts, target_options=opts)

In [ ]:
def gen_refs(f):
    with fs.open(f, **so) as infile:
        h5chunks = SingleHdf5ToZarr(infile, f, inline_threshold=300)
        return h5chunks.translate()  

In [ ]:
cluster = LocalCluster(threads_per_worker=1)
client = Client(cluster)

In [ ]:
%%time
ref_list = dask.compute(*[dask.delayed(gen_refs)(f) for f in flist], retries=10)

In [ ]:
ds = xr.open_dataset(ref_list[0], engine="kerchunk", chunks={}, storage_options=so)

In [ ]:
identical_dims = []
for v in ds.variables.keys():
    if 'time' not in ds[v].dims:
        identical_dims.append(v)

In [ ]:
identical_dims

In [ ]:
combined_parquet = f'combined_nc_{region}.parq'

In [ ]:
fs_local = fsspec.filesystem("file")
try: 
    fs_local.rm(combined_parquet, recursive=True)
except:
    pass

In [ ]:
fs_local.makedirs(combined_parquet)

In [ ]:
out = LazyReferenceMapper.create(root=combined_parquet, fs=fs_local, record_size=100000)

Kerchunk is bombing out on the last set of refs - raised issue here:
    

In [ ]:
%%time
original_refs = MultiZarrToZarr(
        ref_list[:-2],
        remote_protocol="abfs",
        remote_options=opts,
        concat_dims=["time"],
        coo_map={"time": "cf:time"},
        identical_dims=identical_dims,
        out=out).translate()

In [ ]:
out.flush()

try at appending...

Above Doesn't work.  Key error. 

#### Test local parquet before uploading refs

In [ ]:
ds = xr.open_dataset(combined_parquet, engine="kerchunk", chunks={},
            storage_options=dict(remote_protocol='abfs', remote_options=opts, target_options=opts, lazy=True))

In [ ]:
ds

In [ ]:
combined_parquet_abfs = f'abfs://cmre-cc01/data/era5/{combined_parquet}'

In [ ]:
_ = fs.upload(combined_parquet, combined_parquet_abfs, recursive=True)

In [ ]:
fs_local.rm(combined_parquet, recursive=True)

#### Test Parquet refs on Azure blob storage

In [ ]:
ds = xr.open_dataset(combined_parquet_abfs, engine="kerchunk", chunks={},
            storage_options=dict(remote_protocol='abfs', remote_options=opts, target_options=opts, lazy=True))

In [ ]:
ds

In [ ]:
%%time
da = ds['2m_temperature'][:,100,200].load()

In [ ]:
da

In [ ]:
da.plot()